In [1]:
import os
import lzma
import tarfile
import tqdm
import pyarrow as pa
import pyarrow.parquet as pq
import shutil
import pandas as pd

In [ ]:


for path in os.listdir("openwebtext"):
    if not path.endswith(".xz"):
        # delete the file or directory
        full_path = os.path.join("openwebtext", path)
        if os.path.isdir(full_path):
            shutil.rmtree(full_path)
        else:
            os.remove(full_path)

In [ ]:


root = "openwebtext"
xz_files = [f for f in os.listdir(root) if f.endswith(".xz")]

# Parquet writer ----------------------------------------------------------------
parquet_path = os.path.join(root, "openwebtext.parquet")
schema = pa.schema([
    ("source", pa.string()),   # filename inside archive or any ID you like
    ("text",   pa.string())    # the document itself
])
writer = pq.ParquetWriter(parquet_path, schema, compression="zstd")

# Helper for batching -----------------------------------------------------------
BATCH_SIZE = 1_000
buffer = {"source": [], "text": []}

def flush():
    """Write the current buffer to parquet and clear it."""
    if buffer["text"]:                      # non-empty
        table = pa.Table.from_pydict(buffer, schema=schema)
        writer.write_table(table)
        buffer["source"].clear()
        buffer["text"].clear()

# Main extraction loop ----------------------------------------------------------
for xz_file in tqdm.tqdm(xz_files, desc="xz archives"):
    xz_path = os.path.join(root, xz_file)

    # 1. stream-decompress the .xz
    with lzma.open(xz_path) as lzma_file:
        # 2. open the tar stream
        with tarfile.open(fileobj=lzma_file) as tar:
            for member in tar.getmembers():
                if member.isfile() and member.name.endswith(".txt"):
                    member_file = tar.extractfile(member)
                    if member_file is None:
                        continue

                    # 3. read bytes -> decode -> append to buffer
                    txt = member_file.read().decode("utf-8", errors="ignore")
                    buffer["source"].append(
                        f"{xz_file[:-3]}_{os.path.basename(member.name)}"
                    )
                    buffer["text"].append(txt)

                    # 4. flush every BATCH_SIZE rows
                    if len(buffer["text"]) >= BATCH_SIZE:
                        flush()

# final flush, close parquet ----------------------------------------------------
flush()
writer.close()

print(f"Finished. All texts are inside: {parquet_path}")

xz archives: 100%|██████████| 1000/1000 [03:13<00:00,  5.17it/s]

Finished. All texts are inside: openwebtext\openwebtext.parquet


In [8]:
import pandas as pd

In [ ]:
# import pandas as pd
# import os
# import math
# import psutil

# # Load your full dataset
# df = pd.read_parquet("openwebtext/openwebtext.parquet")

# # Estimate size in memory
# mem_bytes = df.memory_usage(deep=True).sum()
# mem_mb = mem_bytes / 1024**2
# print(f"Loaded size: {mem_mb:.2f} MB")

# # Define target size per split (adjust to taste, baddie)
# target_mb = 300
# n_splits = math.ceil(mem_mb / target_mb)

# # Split and save
# os.makedirs("openwebtext/splits", exist_ok=True)

# chunk_size = math.ceil(len(df) / n_splits)

# for i in range(n_splits):
#     start = i * chunk_size
#     end = min((i + 1) * chunk_size, len(df))
#     chunk = df.iloc[start:end]
#     out_path = f"openwebtext/splits/openwebtext_part_{i:02d}.parquet"
#     chunk.to_parquet(out_path, index=False)
#     print(f"Saved {out_path} with {len(chunk)} rows")


Loaded size: 3400.46 MB
Saved openwebtext/splits/openwebtext_part_00.parquet with 32352 rows
Saved openwebtext/splits/openwebtext_part_01.parquet with 32352 rows
Saved openwebtext/splits/openwebtext_part_02.parquet with 32352 rows
Saved openwebtext/splits/openwebtext_part_03.parquet with 32352 rows
Saved openwebtext/splits/openwebtext_part_04.parquet with 32352 rows
Saved openwebtext/splits/openwebtext_part_05.parquet with 32352 rows
Saved openwebtext/splits/openwebtext_part_06.parquet with 32352 rows
Saved openwebtext/splits/openwebtext_part_07.parquet with 32352 rows
Saved openwebtext/splits/openwebtext_part_08.parquet with 32352 rows
Saved openwebtext/splits/openwebtext_part_09.parquet with 32352 rows
Saved openwebtext/splits/openwebtext_part_10.parquet with 32352 rows
Saved openwebtext/splits/openwebtext_part_11.parquet with 32350 rows


In [5]:
df = pd.read_parquet(r"../../data/tokenized/owt_part_00.parquet")

In [6]:
df['text'].apply(len).sum()

np.int64(38133042)

In [27]:
for path in os.listdir("owt"):
    if path.endswith(".parquet"):
        df = pd.read_parquet(f"owt/{path}")
        df['text'] = df['text'].apply(clean_text)
        df.to_parquet(f"owt/{path}")

In [9]:
df['text'].apply(len).mean()

np.float64(1178.6919510385758)

In [33]:
new_df = df[df['text'].apply(len) <= 2096]

In [34]:
new_df['text'].apply(len).mean()

np.float64(786.1747071463228)

In [35]:
new_df['text'].apply(len).sum()

np.int64(22684285)

In [36]:
12_163_521

12163521

In [29]:
def get_bin(text):
    return len(text)//16

In [30]:
new_df['bin'] = new_df['text'].apply(lambda x: len(x)//16)

C:\Users\luequ\AppData\Local\Temp\ipykernel_19212\1360437756.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['bin'] = new_df['text'].apply(lambda x: len(x)//16)


In [31]:
for b in new_df['bin'].value_counts():
    print(b)










557
530
530
529
527
525
516
516
514
512
509
507
502
499
496
491
473
460
451
451
440
432
418
407
405
385
382
382
369
360
351
349
347
337
335
328
328
317
304
296
294
290
290
278
269
269
259
258
258
248
241
226
221
217
116
100
22
2
2


In [ ]:
from datasets import load_dataset

dataset = load_dataset("Skylion007/openwebtext")


c:\Users\luequ\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\luequ\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\luequ\.cache\huggingface\hub\datasets--Skylion007--openwebtext. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an

RuntimeError: Dataset scripts are no longer supported, but found openwebtext.py

In [ ]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="Skylion007/openwebtext",
    repo_type="dataset",
    local_dir="openwebtext_dataset",  # or wherever you want it saved
    local_dir_use_symlinks=False  # make it an actual copy
)

c:\Users\luequ\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:980: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(
Fetching 24 files:   4%|▍         | 1/24 [00:00<00:15,  1.53it/s]